# Import Libraries

In [ ]:
!pip uninstall -y numpy
!pip install numpy --upgrade



Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 41.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is incompatible.


In [ ]:
!pip uninstall -y scipy gensim
!pip install scipy==1.10.1 gensim==4.3.1


Found existing installation: scipy 1.14.1
Uninstalling scipy-1.14.1:
  Successfully uninstalled scipy-1.14.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 103.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cvxpy 1.6.4 requires scipy>=1.11.0, but you have scipy 1.10.1 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompat

In [ ]:
!pip install numpy==1.23.5


In [ ]:
!pip install --upgrade gensim
!pip install --upgrade scipy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 51.4 MB/s eta 0:00:00
  Attempting uninstall: gensim
    Found existing installation: gensim 4.3.1
    Uninstalling gensim-4.3.1:
      Successfully uninstalled gensim-4.3.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 16.0 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.15.2 which is incompatible.
pymc 5.21.2 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
imbalanced-learn 0.13.0 requires numpy<3,>=1.24.3, but you have numpy 1.23.5 which is incompatible.
albume

#  Loading Pre-trained Embeddings:

In [ ]:
import gensim.downloader as api

print("Loading Word2Vec model...")
word2vec_model = api.load("word2vec-google-news-300")
print("Word2Vec model loaded!")

print("Loading GloVe model...")
glove_model = api.load("glove-wiki-gigaword-300")
print("GloVe model loaded!")


Loading Word2Vec model...
[==================================================] 100.0% 1662.8/1662.8MB downloaded
Word2Vec model loaded!
Loading GloVe model...
[==================================================] 100.0% 376.1/376.1MB downloaded
GloVe model loaded!


# Compute Cosine Similarity

In [ ]:
import numpy as np

def compute_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2) + 1e-10)


# Retrieve word or phrase embedding from a pretrained model

In [ ]:
def get_word_vector(word, model):
    if word in model:
        return model[word]
    elif " " in word:
        parts = word.split(" ")
        vectors = [model[p] for p in parts if p in model]
        if vectors:
            return np.mean(vectors, axis=0)
    return None

# Find the most similar word to a given vector

In [ ]:
def find_closest_word(model, target_vector, ignore_set):
    best_similarity = -1
    best_match = None
    for word in model.index_to_key:
        if word in ignore_set:
            continue
        similarity = compute_similarity(model[word], target_vector)
        if similarity > best_similarity:
            best_similarity = similarity
            best_match = word
    return best_match

# Evalute the models

In [ ]:
def run_analogy_test(model, analogy_data, model_label="My Word Embedding Model"):
    total = 0
    correct = 0
    print(f"\nRunning analogy evaluation for: {model_label}\n")

    for a, b, c, expected in analogy_data:
        a_vec = get_word_vector(a, model)
        b_vec = get_word_vector(b, model)
        c_vec = get_word_vector(c, model)

        if any(v is None for v in (a_vec, b_vec, c_vec)):
            print(f"{a}:{b} :: {c}:? --> Skipped (missing vector)")
            continue

        target_vec = b_vec + c_vec - a_vec
        predicted = find_closest_word(model, target_vec, {a, b, c})

        is_correct = predicted == expected
        print(f"{a}:{b} :: {c}:? --> Predicted: {predicted} | Expected: {expected}")

        total += 1
        if is_correct:
            correct += 1

    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"\nFinal accuracy: {accuracy:.2f}%")


# Designing My Own Word Analogy Tests

In [ ]:
analogies_sem = [
  ("milk", "cow", "honey", "bee"),
("light", "dark", "day", "night"),
("mountain", "hill", "ocean", "sea"),
("book", "author", "song", "singer"),
("compass", "direction", "clock", "time"),
("oxygen", "breathe", "food", "eat"),
("programming", "it", "medicine", "pharmacy"),
("glasses", "eyes", "headphones", "ears"),
("coffee", "caffeine", "beer", "alcohol"),
("doctor", "hospital", "teacher", "school")
]



In [ ]:
analogies_syn = [
    ("jump", "jumped", "sing", "sang"),
    ("bright", "brightness", "happy", "happiness"),
    ("break", "broken", "go", "gone"),
    ("educate", "education", "communicate", "communication"),
    ("test", "tested", "train", "trained"),
    ("try", "tries", "cry", "cries"),
    ("teach", "teacher", "build", "builder"),
    ("easy", "easier", "hard", "harder"),
    ("boy", "boys", "girl", "girls"),
    ("honest", "honestly", "calm", "calmly"),

]


In [ ]:
run_analogy_test(word2vec_model, analogies_sem, "Word2Vec")
run_analogy_test(glove_model, analogies_sem, "GloVe")
run_analogy_test(word2vec_model, analogies_syn, "Word2Vec")
run_analogy_test(glove_model, analogies_syn, "GloVe")



Running analogy evaluation for: Word2Vec

milk:cow :: honey:? --> Predicted: bee | Expected: bee
light:dark :: day:? --> Predicted: night | Expected: night
mountain:hill :: ocean:? --> Predicted: sea | Expected: sea
book:author :: song:? --> Predicted: songs | Expected: singer
compass:direction :: clock:? --> Predicted: tackled_inbounds | Expected: time
oxygen:breathe :: food:? --> Predicted: eat | Expected: eat
programming:it :: medicine:? --> Predicted: medicines | Expected: pharmacy
glasses:eyes :: headphones:? --> Predicted: earphones | Expected: ears
coffee:caffeine :: beer:? --> Predicted: caffeinated_drinks | Expected: alcohol
doctor:hospital :: teacher:? --> Predicted: elementary | Expected: school

Final accuracy: 40.00%

Running analogy evaluation for: GloVe

milk:cow :: honey:? --> Predicted: mad | Expected: bee
light:dark :: day:? --> Predicted: days | Expected: night
mountain:hill :: ocean:? --> Predicted: waters | Expected: sea
book:author :: song:? --> Predicted: singer